In [3]:
import random
import math

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML


In [8]:
# Generate some fake data.

def f(x, a=0, b=0):
  return np.multiply(a, x) + b

real_a, real_b = (0.7, 0.3)

x = np.random.uniform(0.1, 4.9, size=25)
y = f(x, a=real_a, b=real_b) + np.random.uniform(-0.025, 0.025, size=25)


array([0.6226122 , 3.64479056, 1.96034792, 1.90718833, 0.96107154,
       3.71149974, 1.2827879 , 2.63348418, 0.91192681, 0.65636866,
       1.53369641, 3.07514029, 0.47455737, 1.71305954, 3.23828451,
       1.65430981, 3.62503262, 3.14910678, 3.40987375, 2.62449821,
       2.4863592 , 1.36195604, 0.6680317 , 1.1999587 , 3.43359442])

**My Cost Function** (it is different from SVM cost)

$cost(x, y) = \frac{1}{N}\sum (y-(a^T x+b))^2 + \lambda a^T a$

**Gradients with respect to a,b**

$\nabla_a cost(x, y) = \left [\frac{1}{N}\sum -2x \cdot (y-(a^T x+b)) \right ] + \lambda a$

$\nabla_b cost(x, y) = \left [\frac{1}{N}\sum -2 \cdot (y-(a^T x+b))  \right ]$

In [23]:
# x: feature vector
# y: true label value
# a, b: learned parameters
#
# Returns: Cost of 1/N sum [ (y - (ax+b))^2]
#
# This is not the SVM cost function.  It is simple linear regression but
# conceptually gradient descent will work the same way.
#
# This code assumes a & x are scalars, i.e. we have only a single feature.
# Everything below would need to be slightly rewriten to handle multiple
# features.
def cost_function(x, y, a, b):
  return np.mean( np.square( y - f(x, a, b) ))
  

# x: feature vector
# y: true label value
# a, b: current estimate of learned parameters
def gradient_of_cost(x, y, a, b, lam=0):
  print(y)
  # Gradient w.r.t. a = Cost of 1/N sum [ (y - (ax+b))^2]
  gradient_a = np.mean( np.multiply(-2, np.multiply((y - f(x, a, b)), x))) + np.multiply(lam, a)
  gradient_b = np.mean( np.multiply(-2, (y - f(x, a, b)) ))
  
  return [gradient_a, gradient_b]

In [11]:
#@title Animation code.  Totally unrelated to SGD and SVMs.
# 

def animate(all_a_est, all_b_est, all_costs):
  # Uses globals x, y, f, cost_function, etc.  Bad.  
  
  a_range = (np.min(all_a_est)-0.1, np.max(all_a_est)+0.1)
  b_range = (np.min(all_b_est)-0.1, np.max(all_b_est)+0.1)
  
  costs_range = (np.min(all_costs), np.max(all_costs)*1.05)
  
  # First set up the figure, the axes, and the plot element
  fig, all_ax = plt.subplots(1,3,figsize=(18,5))
  plt.close()

  # Parameter Space graph
  ax = all_ax[0]

  ax.set_xlim(a_range)
  ax.set_ylim(b_range)
  ax.set_xlabel("a")
  ax.set_ylabel("b")
  ax.set_title("Parameter space (a & b)")

  init_lines = []
  init_lines.append(ax.plot(real_a, real_b, 'go'))

  line2, = ax.plot([], [], lw=2)
  
  # Cost Graph
  ax = all_ax[1]

  ax.set_xlim((0, len(all_costs)))
  ax.set_ylim(costs_range)
  
  ax.set_xlabel("Iteration")
  ax.set_ylabel("Log(Cost(Training))")
  ax.set_title("Cost Function")

  cost_line, = ax.plot([], [])
  
  # ax+b and data
  ax = all_ax[2]
  
  init_lines.append(ax.plot(x, y, 'bo'))
  line3, = ax.plot([], [], lw=2)
  
  # Hardcode data limits cause I am lazy.
  ax.set_xlim((0, 5))
  ax.set_ylim((0,4))
  
  ax.set_xlabel("x")
  ax.set_ylabel("y (i.e. label)")
  ax.set_title("Estimate = ax+b")
  

  # initialization function: plot the background of each frame
  def init():    
      return init_lines

  # animation function: this is called sequentially
  def update_frame(i):
    line2.set_data(all_a_est[0:i], all_b_est[0:i])
    cost_line.set_data(np.arange(0,i), all_costs[0:i])
    line3.set_data([0,5], [all_b_est[i], all_a_est[i]*5+all_b_est[i]])
    return (line2,cost_line,)

  anim = animation.FuncAnimation(fig, update_frame, init_func=init, frames=len(all_a_est), interval=100, blit=False)

  rc('animation', html='jshtml')
  return anim

In [12]:
#@title Animation Test Code.  Ignore.
animate([0,1,0.5,.6,.3], [1,.3,.4,.2,.9],[0,1,2,1,3])  # Test case for animation code.

ValueError: Key animation.html: Unrecognized animation.html string "jshtml": valid strings are ['html5', 'none']

In [13]:
def step(x_batch, y_batch ,a_est, b_est, lam=0, eta=1):
  c = cost_function(x_batch,y_batch,a=a_est,b=b_est)
  g = gradient_of_cost(x_batch,y_batch,a=a_est,b=b_est,lam=lam)
  
  a_new = a_est - eta * g[0]
  b_new = b_est - eta * g[1]
  
  #print ("a,b = %f, %f.  C = %f.  Ga,b = %f, %f.  New a,b = %f, %f." % (a_est, b_est, c, g[0], g[1], a_new, b_new))
  
  return (a_new, b_new)

In [16]:
def stochastic_gradient_descent(x, y, initial_a, initial_b, eta):
  a_est = initial_a
  b_est = initial_b

  # Lambda
  lam = 0

  all_a_est = [a_est]
  all_b_est = [b_est]
  all_costs = [cost_function(x,y,a=a_est,b=b_est)]

  for i in range(50):
    n = random.randint(0, len(x)-1) # Batch size of 1.
    a_est, b_est = step(x[n], y[n], a_est, b_est, lam, eta)

    # Normally we wouldn't collect cost and parameters every iteration, 
    # but this is a very simple function to learn.
    all_a_est.append(a_est)
    all_b_est.append(b_est)
    # This is the batch training cost...this should be fixed to be validation set.
    all_costs.append(math.log(cost_function(x,y,a=a_est,b=b_est)))

  print("Final a,b = %f, %f." % (a_est, b_est))
  
  return a_est, b_est, all_a_est, all_b_est, all_costs

In [24]:
_, _, all_a_est, all_b_est, all_costs = stochastic_gradient_descent(x, y, 0, 0, eta=0.04)

1.361956037513654
3.644790564144076
3.149106778153701
3.4335944207582028
2.633484181825752
3.238284505656513
1.7130595418201584
3.625032618829647
0.6226121998158408
3.238284505656513
0.6226121998158408
1.7130595418201584
3.4098737543769766
3.0751402902619556
3.149106778153701
2.624498214055936
1.361956037513654
0.6226121998158408
3.0751402902619556
2.624498214055936
0.4745573691321877
1.361956037513654
1.5336964089452396
1.5336964089452396
3.644790564144076
3.4098737543769766
0.6226121998158408
0.9610715447251874
3.625032618829647
3.149106778153701
1.6543098141216355
0.9119268095422202
3.0751402902619556
3.238284505656513
0.9610715447251874
1.361956037513654
0.6226121998158408
3.0751402902619556
2.624498214055936
2.4863591958454947
3.4335944207582028
3.0751402902619556
1.9603479199071878
0.9119268095422202
0.6563686553768513
3.238284505656513
3.0751402902619556
0.4745573691321877
3.4098737543769766
1.6543098141216355
Final a,b = 0.710085, 0.259354.


In [30]:
_, _, all_a_est, all_b_est, all_costs = stochastic_gradient_descent(x, y, 0, 0, eta=0.11) # eta too big.
animate(all_a_est, all_b_est, all_costs)

Output hidden; open in https://colab.research.google.com to view.

In [31]:
_, _, all_a_est, all_b_est, all_costs = stochastic_gradient_descent(x, y, 0, 0, eta=0.001) # eta too low.
animate(all_a_est, all_b_est, all_costs)

Output hidden; open in https://colab.research.google.com to view.